# Objective

1. Point to Zones



In [6]:
import pandas as pd

import folium

import geopandas as gp

import numpy as np



from scipy.spatial import ConvexHull

import random

from shapely.geometry import Point, Polygon

import matplotlib.pyplot as plt

import networkx as nx

import math

# from osgeo import ogr


In [7]:
def Check_zones(pnt,polys):
    
    key='None'
    
    for k, geom in polys.items():
        
        if pnt.within(geom):
            
            key=k
            
            break
            
    return key

In [8]:
'''Load data'''

'''Zone-related data'''

Taxi_Zones=np.load('./Data/NYC_Zones/Taxi_Zones.npy',allow_pickle=True).item()

Zone_list=np.load('./Data/NYC_Zones/Zone_list.npy',allow_pickle=True)

Zone_Center=np.load('./Data/NYC_Zones/Zone_Center.npy',allow_pickle=True).item()

Zone_Link=np.load('./Data/NYC_Zones/Zone_Link.npy',allow_pickle=True).item()

'''Link-related data'''

Link_geometry=np.load('./Data/NYC_Zones/Link_geometry.npy',allow_pickle=True).item()

'''GeoSeries Object'''

polys = gp.GeoSeries(Taxi_Zones)


In [9]:
'''2. Constructing Road Network'''

Street_data = gp.read_file('./Data/NYC_Street/geo_export_2d0a9cb9-782e-42c0-ba15-7345a0f64002.shp')

Street_data=Street_data.loc[(Street_data['borocode']=='1')&(Street_data['rw_type'].isin([1,3]))&(Street_data['status']=='2')]

Street_data=Street_data[['st_name','geometry','shape_leng']]

Street_data=Street_data.rename(columns={'shape_leng':'length'})

Street_data['start']=Street_data.apply(lambda x:Point(list(x['geometry'].coords)[0]),axis=1)

Street_data['end']=Street_data.apply(lambda x:Point(list(x['geometry'].coords)[-1]),axis=1)

Street_data['start_zone']=Street_data.apply(lambda x:Check_zones(x['start'],polys),axis=1)

Street_data=Street_data.loc[Street_data['start_zone']!='None']

Street_data['end_zone']=Street_data.apply(lambda x:Check_zones(x['end'],polys),axis=1)

Street_data=Street_data.loc[Street_data['end_zone']!='None']                        
                        
Street_data=Street_data.reset_index(drop=True)

Street_data['Link_id']=['Link_'+str(i) for i in Street_data.index]

Street_data=Street_data[['Link_id','st_name','geometry','length','start','start_zone','end','end_zone']]


'''Encode'''

Get_Str=lambda x,pos:str(list(x[pos].coords)[0][0])+'&'+str(list(x[pos].coords)[0][1])

Street_data['start_str']=Street_data.apply(lambda x:Get_Str(x,'start'),axis=1)

Street_data['end_str']=Street_data.apply(lambda x:Get_Str(x,'end'),axis=1)

'''Point list'''

Ponits_str=list(set(list(Street_data['start_str'])+list(Street_data['end_str'])))

Ponits_dic={Ponits_str[i]:'Point_'+str(i) for i in range(len(Ponits_str))}

Points_list=list(Ponits_dic.values())

'''Look up'''

Street_data['start_point']=Street_data.apply(lambda x:Ponits_dic[x['start_str']],axis=1)

Street_data['end_point']=Street_data.apply(lambda x:Ponits_dic[x['end_str']],axis=1)

Street_data=Street_data[['Link_id','st_name','geometry','length',\
                         'start','start_zone','end','end_zone',\
                        'start_point','end_point']]

'''Save the processed data'''

Point_coordinate={}

Link_Point={}

Point_link={pnt:list() for pnt in Points_list}

Zone_Point={zone:list() for zone in Zone_list}

Point_zone={}

for idx,row in Street_data.iterrows():
    
    Link_Point[row['Link_id']]=(row['start_point'],row['end_point'],row['length'])
    
    if row['Link_id'] not in Point_link[row['start_point']]:
        Point_link[row['start_point']].append(row['Link_id'])
    if row['Link_id'] not in Point_link[row['end_point']]:
        Point_link[row['end_point']].append(row['Link_id'])
    
    if row['start_point'] not in Point_coordinate.keys():
        Point_coordinate[row['start_point']]=row['start']
        Point_zone[row['start_point']]=row['start_zone']
    if row['end_point'] not in Point_coordinate.keys():
        Point_coordinate[row['end_point']]=row['end']
        Point_zone[row['end_point']]=row['end_zone']
        
    if row['start_point'] not in Zone_Point[row['start_zone']]:
        Zone_Point[row['start_zone']].append(row['start_point'])
    if row['end_point'] not in Zone_Point[row['end_zone']]:
        Zone_Point[row['end_zone']].append(row['end_point'])
    
        
        
        
np.save('./Data/NYC_Zones/Points_list',Points_list)

np.save('./Data/NYC_Zones/Point_coordinate',Point_coordinate)
 
np.save('./Data/NYC_Zones/Link_Point',Link_Point)



np.save('./Data/NYC_Zones/Point_link',Point_link)

np.save('./Data/NYC_Zones/Zone_Point',Zone_Point)

np.save('./Data/NYC_Zones/Point_zone',Point_zone)


'''Zone_Point,Point_coordinate,Link_Point'''




'Zone_Point,Point_coordinate,Link_Point'

In [11]:
'''4. Designing a method for the shortest path'''

'''Graph'''

G = nx.Graph()

G.add_nodes_from(Points_list)

G.add_weighted_edges_from(list(Link_Point.values()))

def Get_path(G,source,target,Point_coordinate):
    
    link_path=list()
    
    try:
    
        path=nx.shortest_path(G, source=source, target=target,weight='weight')

        shortest_dis=nx.shortest_path_length(G, source=source, target=target,weight='weight')
        
        link_path=path

        
    except:
        
        '''Path'''
        
        start_lng,start_lat=list(Point_coordinate[source].coords)[0][0],list(Point_coordinate[source].coords)[0][1]
        
        end_lng,end_lat=list(Point_coordinate[target].coords)[0][0],list(Point_coordinate[target].coords)[0][1]
        
        '''10 parts'''
        
        
        for i in range(1,10,1):
            
            pnt_lng=start_lng+(end_lng-start_lng)*(i/10)
            
            pnt_lat=start_lat+(end_lat-start_lat)*(i/10)
            
            pnt=Point(pnt_lng,pnt_lat)
            
            zone=Check_zones(pnt,polys)
            
            point=Check_points(pnt,zone,Point_coordinate)
            
            path_points.append(point)
               
        link_path=[source]+list(set(path_points))+[target]
            

        '''Distance'''
        
        shortest_dis=Point_coordinate[source].distance(Point_coordinate[target])*111000
    
        
    return link_path,shortest_dis




link_path,shortest_dis=Get_path(G,'Point_6','Point_10',Point_coordinate)

link_path


['Point_6',
 'Point_5579',
 'Point_1787',
 'Point_2225',
 'Point_207',
 'Point_4816',
 'Point_4947',
 'Point_4634',
 'Point_5090',
 'Point_5956',
 'Point_10']

In [14]:
'''Visualization'''

df=pd.DataFrame(np.array(list(Taxi_Zones.values())),columns=['geometry'])

df=gp.GeoDataFrame(df)

df=df.set_crs(epsg=4326)

'''Background'''

background = folium.Map([40.769602, -73.973667],tiles='openstreetmap',zoom_start=12)

'''Zones'''

folium.Choropleth(
    geo_data=df,
    fill_color='blue',
    fill_opacity=0.1,
    name='Zone').add_to(background)

'''Points demo'''

for pnt in Zone_Point['Zone_12']:
    
    for link in Point_link[pnt]:
        
        folium.PolyLine([x[::-1] for x in list(Link_geometry[link].coords)],\
                    color="red",\
                    weight=2.0,\
                    opacity=0.6).add_to(background)
        
    folium.Circle(
    radius=10,
    location=list(Point_coordinate[pnt].coords)[0][::-1],
    color='Black',
    fill=True,
    ).add_to(background)
    
    


folium.LayerControl().add_to(background)


background